<a href="https://colab.research.google.com/github/mariyakukhtinova-sys/dbs/blob/main/Maria_Kukhtinova_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
pip install pyspark

In [4]:
pip install findspark

In [5]:
import findspark
findspark.init()

In [6]:
import pyspark
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
spark = (
    SparkSession
    .builder
    .appName('Test_01')
    .config('spark.ui.port', '9311')
    .config('spark.executor.memoryOverhead', '1G')
    .config('spark.shuffle.service.enabled', 'true')
    .config('spark.dynamicAllocation.enabled', 'true')
    .config('spark.driver.extraClassPath', '/opt/spark/jars/sqljdbc42.jar')\
    .config('spark.executor.extraClassPath', '/opt/spark/jars/sqljdbc42.jar')\
    .getOrCreate()
)

In [7]:
spark

In [8]:
from pyspark.sql import functions as f

## Первый датафрейм с данными по отелю

In [30]:
logs_hotel = spark.read.option("header",True).option("sep",",").csv("Hotel.csv")

In [10]:
logs_hotel.show()

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|2017|   10|   2|       Offline|             0|                     0|                             0|            65|       

In [31]:
logs_hotel.printSchema()

root
 |-- ID: string (nullable = true)
 |-- n_adults: string (nullable = true)
 |-- n_children: string (nullable = true)
 |-- weekend_nights: string (nullable = true)
 |-- week_nights: string (nullable = true)
 |-- meal_plan: string (nullable = true)
 |-- car_parking_space: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- lead_time: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- date: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- repeated_guest: string (nullable = true)
 |-- previous_cancellations: string (nullable = true)
 |-- previous_bookings_not_canceled: string (nullable = true)
 |-- avg_room_price: string (nullable = true)
 |-- special_requests: string (nullable = true)
 |-- status: string (nullable = true)



Все данные типа string.

Так не пойдет, укажем свою собственную схему с корректными типами и перечитаем файл


In [32]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, BooleanType

schema = StructType([
    StructField("ID", StringType(), True),
    StructField("n_adults", IntegerType(), True),
    StructField("n_children", IntegerType(), True),
    StructField("weekend_nights", IntegerType(), True),
    StructField("week_nights", IntegerType(), True),
    StructField("meal_plan", StringType(), True),
    StructField("car_parking_space", IntegerType(), True),
    StructField("room_type", StringType(), True),
    StructField("lead_time", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("date", IntegerType(), True),
    StructField("market_segment", StringType(), True),
    StructField("repeated_guest", IntegerType(), True),
    StructField("previous_cancellations", IntegerType(), True),
    StructField("previous_bookings_not_canceled", IntegerType(), True),
    StructField("avg_room_price", DoubleType(), True),
    StructField("special_requests", IntegerType(), True),
    StructField("status", StringType(), True)
])

logs_hotel = spark.read.option("header",True).option("sep",",").schema(schema).csv("Hotel.csv")

logs_hotel.show()

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|      ID|n_adults|n_children|weekend_nights|week_nights|   meal_plan|car_parking_space|  room_type|lead_time|year|month|date|market_segment|repeated_guest|previous_cancellations|previous_bookings_not_canceled|avg_room_price|special_requests|      status|
+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------+----------------------+------------------------------+--------------+----------------+------------+
|INN00001|       2|         0|             1|          2| Meal Plan 1|                0|Room_Type 1|      224|2017|   10|   2|       Offline|             0|                     0|                             0|          65.0|       

In [13]:
logs_hotel.describe().show()

+-------+--------+------------------+-------------------+-----------------+------------------+------------+-----------+-----------------+------------------+------------------+------------------+--------------+----------------------+------------------------------+-----------------+------------------+------------+
|summary|      ID|          n_adults|         n_children|   weekend_nights|       week_nights|   meal_plan|  room_type|        lead_time|              year|             month|              date|market_segment|previous_cancellations|previous_bookings_not_canceled|   avg_room_price|  special_requests|      status|
+-------+--------+------------------+-------------------+-----------------+------------------+------------+-----------+-----------------+------------------+------------------+------------------+--------------+----------------------+------------------------------+-----------------+------------------+------------+
|  count|   36275|             36275|              36275| 

In [33]:
# сразу сделаем колонку с датой
logs_hotel = logs_hotel.withColumn(
    "full_date",
    f.concat_ws("-",
        f.col("year"),
        f.lpad(f.col("month"), 2, "0"),
        f.lpad(f.col("date"), 2, "0")
    ).cast("date")
)


# в данных есть 37 записей за 29 февраля 2018г - некорректная дата
# удалим записи, чтобы можно было работать с датафреймом
logs_hotel = logs_hotel.filter(
    ~(
        (f.col('year') == 2018) &
        (f.col('month') == 2) &
        (f.col('date') == 29)
    )
)

#logs_hotel.show()
#logs_hotel.printSchema()

## Второй датафрейм с календарем

In [15]:
from pyspark.sql.functions import udf
import pandas as pd

start_date = '2017-01-01'
end_date = '2018-12-31'

date_range = pd.date_range(start=start_date, end=end_date, freq='D')

calendar = spark.createDataFrame(
    [(d.date(),) for d in date_range],
    ["calendar_dt"]
)

calendar.show(5)

+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
+-----------+
only showing top 5 rows


## Запросы

    Вычислить среднее количество ночей, которые гости проводят в отеле
    (только для подтвержденных бронирований, с детализацией по месяцам и годам)

In [16]:
logs_hotel.filter(f.col('status') == 'Not_Canceled')\
  .groupBy(f.col('year'), f.col('month'))\
  .agg(f.mean(f.col('weekend_nights') + f.col('week_nights')).alias('avg_total_nights'))\
  .orderBy('year', 'month') \
  .show()

+----+-----+------------------+
|year|month|  avg_total_nights|
+----+-----+------------------+
|2017|    7|3.0166666666666666|
|2017|    8|2.7189384800965017|
|2017|    9|2.6550783912747105|
|2017|   10|2.7032898820608318|
|2017|   11|2.7241935483870967|
|2017|   12| 3.043046357615894|
|2018|    1|2.7414141414141415|
|2018|    2|2.6768488745980705|
|2018|    3|3.0392038600723765|
|2018|    4| 2.924755887421022|
|2018|    5|2.8054545454545456|
|2018|    6| 2.596757322175732|
|2018|    7|3.1938088829071334|
|2018|    8|3.1544117647058822|
|2018|    9| 2.786425902864259|
|2018|   10|2.8910186199342824|
|2018|   11|2.9784511784511785|
|2018|   12|3.2521891418563924|
+----+-----+------------------+




    Определить ТОП-3 месяца по проценту отмененных броней за 2018 год.


In [17]:
logs_hotel\
  .filter(f.col('year') == 2018)\
  .groupBy(f.col('month'))\
  .agg((100.0 * f.count(f.when(f.col('status') == 'Canceled', 1)) / f.count('*')).alias('cancel_rate_prct'))\
  .orderBy(f.col('cancel_rate_prct'), ascending = False)\
  .show(3)

+-----+-----------------+
|month| cancel_rate_prct|
+-----+-----------------+
|    8|46.55234012147196|
|   10| 46.3572267920094|
|    9|45.77987846049966|
+-----+-----------------+
only showing top 3 rows


    Вычислить среднее время на каждый месяц между бронированием и заездом в отель для подтвержденных броней.


In [18]:
logs_hotel.filter(f.col('status') == 'Not_Canceled')\
  .groupBy(f.col('month'))\
  .agg(f.mean(f.col('lead_time')).alias('avg_lead_days'))\
  .orderBy(f.col('month'))\
  .show()

+-----+------------------+
|month|     avg_lead_days|
+-----+------------------+
|    1| 34.87171717171717|
|    2|30.272508038585208|
|    3| 43.19420989143546|
|    4|62.492820218265365|
|    5| 60.98909090909091|
|    6| 70.63546025104603|
|    7| 90.15691158156912|
|    8|  65.9694623655914|
|    9| 57.78490074845428|
|   10| 65.10736107070119|
|   11| 41.02232779097387|
|   12|61.794196258113786|
+-----+------------------+



    Вычислить общую среднюю выручку на каждый месяц в каждом году, сгруппировав по всем типам бронирования для подтвержденных броней, и вывести это в виде сводной таблицы (PIVOT).


In [19]:
avg_revenue = logs_hotel\
    .filter(f.col('status') == 'Not_Canceled')\
    .groupBy('year' , 'month',)\
    .pivot('market_segment')\
    .agg(f.avg('avg_room_price').alias('avg_revenue'))\
    .orderBy('year', 'month')\
    .show()

+----+-----+------------------+-------------------+------------------+------------------+------------------+
|year|month|          Aviation|      Complementary|         Corporate|           Offline|            Online|
+----+-----+------------------+-------------------+------------------+------------------+------------------+
|2017|    7|              NULL|  7.465999999999999|              65.0| 88.18870129870129| 65.55323529411764|
|2017|    8|              NULL|0.32432432432432434| 66.91666666666667| 97.16254355400697| 97.85685144124166|
|2017|    9|              NULL|   7.37037037037037| 86.03361344537815| 96.79672910662809|113.32639553429031|
|2017|   10|              NULL| 0.5454545454545454| 84.48039215686275| 89.08058751529984| 101.9722835820896|
|2017|   11|              NULL|             4.9375| 68.08036144578314| 66.17945255474449| 84.75230769230772|
|2017|   12|              NULL|               0.05| 69.02763440860215|  69.9498342541436| 87.31105011933171|
|2018|    1|       

    Выявить ТОП-5 постоянных гостей, которые принесли наибольшую выручку за все время, и показать их долю в общей выручке от постоянных гостей. Использовать уникальный идентификатор брони как уникальный идентификатор гостя, предположив, что 1 бронь = 1 гость.


In [51]:
from pyspark.sql.window import Window

logs_hotel\
  .filter(f.col('repeated_guest') == 1)\
  .withColumn('booking_revenue',(f.col('week_nights') + f.col('weekend_nights')) * f.col('avg_room_price'))\
  .groupBy('ID')\
  .agg(f.sum('booking_revenue').alias('total_revenue'))\
  .withColumn('total_revenue_all_repeated',
                f.sum('total_revenue').over(Window.partitionBy()))\
  .withColumn('share_of_total', f.col('total_revenue') / f.col('total_revenue_all_repeated') * 100)\
  .orderBy('share_of_total', ascending = False)\
  .show(5)

+--------+------------------+--------------------------+------------------+
|      ID|     total_revenue|total_revenue_all_repeated|    share_of_total|
+--------+------------------+--------------------------+------------------+
|INN19235|1754.3999999999999|        115032.89999999997|1.5251288979065993|
|INN05222|             690.0|        115032.89999999997|0.5998283969194902|
|INN14189|             665.0|        115032.89999999997|0.5780954839876247|
|INN09923|             660.0|        115032.89999999997|0.5737489014012515|
|INN25479|             650.0|        115032.89999999997|0.5650557362285052|
+--------+------------------+--------------------------+------------------+
only showing top 5 rows


    Вывести общее количество гостей на каждый день в отеле, отсортировав по убыванию дат, включая дни, когда отель пустует. Также рассчитать процент загрузки для каждого дня, если известно, что общая вместимость отеля 400 человек.

In [64]:
tmp = logs_hotel\
  .groupBy("full_date")\
  .agg(
      (f.sum("n_adults") + f.sum("n_children")).alias("guests"),
      (100.0*(f.sum("n_adults") + f.sum("n_children"))/400).alias("load_prct")
      )\
  .orderBy("full_date", ascending = False)

calendar.join(tmp, on=calendar["calendar_dt"] == tmp["full_date"], how="left")\
  .select(
        "calendar_dt",
        f.coalesce(f.col("guests"), f.lit(0)).alias("guests"),
        f.coalesce(f.col("load_prct"), f.lit(0.0)).alias("load_prct")
    )\
    .orderBy("calendar_dt", ascending=False)\
  .show()

+-----------+------+---------+
|calendar_dt|guests|load_prct|
+-----------+------+---------+
| 2018-12-31|    89|    22.25|
| 2018-12-30|   212|     53.0|
| 2018-12-29|   216|     54.0|
| 2018-12-28|   183|    45.75|
| 2018-12-27|   325|    81.25|
| 2018-12-26|   236|     59.0|
| 2018-12-25|   114|     28.5|
| 2018-12-24|   119|    29.75|
| 2018-12-23|   152|     38.0|
| 2018-12-22|   109|    27.25|
| 2018-12-21|   106|     26.5|
| 2018-12-20|    73|    18.25|
| 2018-12-19|    93|    23.25|
| 2018-12-18|    87|    21.75|
| 2018-12-17|    91|    22.75|
| 2018-12-16|   166|     41.5|
| 2018-12-15|    66|     16.5|
| 2018-12-14|    50|     12.5|
| 2018-12-13|    61|    15.25|
| 2018-12-12|    56|     14.0|
+-----------+------+---------+
only showing top 20 rows
